In [0]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

1. Definição do indivíduo 


In [0]:
class City:
    def __init__(self, x, y): #construtor, parametro self obrigatório
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"


2. Criar Individuo 


In [0]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

3. Criar População Incial

In [0]:
def initialPopulation(popSize, cityList):
    population = [] #declaraçãão da lista

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

4 Definiçãao Da funçãao de Fitness

In [0]:
#Menor distância, maior será o fitness
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

5. Calculo do fitness

In [0]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

6. Seleção

In [0]:
# usa a saída de rankRota /  elitesize -> seleção com elitismo, que é parte da antiga população 
# que passa diretamente para a nova geração
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults # funçãão de selecao retorna uma lista de rotas IDs

In [0]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

7. cruzamento


In [0]:
def breed(parent1, parent2): #crossover
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [0]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

8. Mutação

In [0]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [0]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

Funçãão que produz a nova geração

In [0]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

Função que define o algoritmo genetico 

In [0]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [21]:
cityList = []
num_city = 25

for i in range(0,num_city):
  newCity = City(x=int(random.random()*200),y = int(random.random()*200))
  cityList.append(newCity)

  print(newCity)
  #plt.scatter(newCity.x,newCity.y)
  #plt.show()

(146,113)
(174,44)
(153,26)
(16,109)
(72,190)
(127,98)
(151,186)
(154,107)
(36,76)
(109,146)
(89,198)
(133,66)
(77,49)
(54,176)
(187,45)
(10,167)
(24,0)
(102,33)
(48,99)
(42,23)
(80,159)
(127,76)
(18,67)
(67,49)
(172,97)


In [22]:
geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)

Initial distance: 2156.456523348285
Final distance: 874.5248200750842


[(153,26),
 (133,66),
 (127,76),
 (102,33),
 (77,49),
 (67,49),
 (42,23),
 (24,0),
 (18,67),
 (36,76),
 (48,99),
 (16,109),
 (10,167),
 (54,176),
 (72,190),
 (89,198),
 (151,186),
 (80,159),
 (109,146),
 (127,98),
 (146,113),
 (154,107),
 (172,97),
 (187,45),
 (174,44)]

In [0]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()